In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler as ss
from sklearn.utils import shuffle

In [2]:
data = pd.read_csv("../water_potability.csv").dropna()[:2000]

data = shuffle(data)
features=data.dropna(axis=0)
# popping out labels
labels = features.pop("Potability")
# converting values into numpy arrays to reshape them
features = np.array(features)
labels = np.array(labels).reshape(-1,1)

print(features.shape, labels.shape) 

(2000, 9) (2000, 1)


In [3]:
# Normalizing the features for faster learning
features = ss().fit_transform(features)
features[0]

array([ 1.55013017,  0.25189316, -0.53691513, -0.61894213, -0.54631861,
       -0.38136582,  0.81396685, -1.38163365,  0.03378706])

In [4]:
features = torch.tensor(features, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32)
# print(features.shape,labels.shape)

In [5]:
model = torch.nn.Sequential(
    torch.nn.Linear(9,6),
    torch.nn.ReLU(),
    torch.nn.Linear(6,1),
    torch.nn.Sigmoid()
)

In [6]:
loss_fn = torch.nn.BCELoss()
opt = torch.optim.SGD(model.parameters(), lr=1e-3)

In [13]:
epochs = 1000000

for epoch in range(epochs):
    preds = model(features)
    loss = loss_fn(preds,labels)

    loss.backward()

    opt.step()
    opt.zero_grad()

    with torch.no_grad():
        if (epoch+1)%(epochs//10) == 0:
            right = 0
            for i in range(len(preds)):
                if preds[i][0] >= 0.5:
                    preds[i][0] = 1
                else:
                    preds[i][0] = 0

                if preds[i][0] == labels[i][0]:
                    right += 1


            print(f"Loss : {loss}")
            print(f"Accuracy : {round(right * 100/ len(preds), 3)}%")

Loss : 0.5541412234306335
Accuracy : 71.4%
Loss : 0.5537266135215759
Accuracy : 71.4%
